In [1]:
import sqlite3
import glob
from pathlib import Path
import pandas as pd

In [2]:
from enum import Enum
import ast

from enum import Enum

class eSubTipo(Enum):
    PROCESSAMENTO = [
        {"codigo": 1, "nome": "Viníferas"},
        {"codigo": 2, "nome": "Americanas e híbridas"},
        {"codigo": 3, "nome": "Uvas de mesa"},
        {"codigo": 4, "nome": "Sem classificação"}
    ]

    IMPORTACAO = [
        {"codigo": 1, "nome": "Vinhos de mesa"},
        {"codigo": 2, "nome": "Espumantes"},
        {"codigo": 3, "nome": "Uvas frescas"},
        {"codigo": 4, "nome": "Uvas passas"},
        {"codigo": 5, "nome": "Suco de uva"}
    ]

    EXPORTACAO = [
        {"codigo": 1, "nome": "Vinhos de mesa"},
        {"codigo": 2, "nome": "Espumantes"},
        {"codigo": 3, "nome": "Uvas frescas"},
        {"codigo": 4, "nome": "Suco de uva"}
    ]

    def get_by_codigo(self, codigo):
        for item in self.value:
            if item["codigo"] == codigo:
                return item
        return None

class eTipo(Enum):
    PRODUCAO = {"codigo": 2, "nome": "Producao", "subtipo": None}
    PROCESSAMENTO = {"codigo": 3, "nome": "Processamento", "subtipo": eSubTipo.PROCESSAMENTO}
    COMERCIALIZACAO = {"codigo": 4, "nome": "Comercializacao", "subtipo": None}
    IMPORTACAO = {"codigo": 5, "nome": "Importacao", "subtipo": eSubTipo.IMPORTACAO}
    EXPORTACAO = {"codigo": 6, "nome": "Exportacao", "subtipo": eSubTipo.EXPORTACAO}

    @property
    def codigo(self):
        return self.value["codigo"]

    @property
    def nome(self):
        return self.value["nome"]

    @property
    def subtipo(self):
        return self.value["subtipo"]

"""
    A classe Negocio define o tipo de análise (Produção, Processamento, Comercialização, Importação, Exportação)
"""
class Negocio():
    
    def __init__(self, negocio:eTipo):
        self.codigo = negocio.codigo
        self.tipo = negocio.nome
        self.subtipo = negocio.subtipo

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type:
            print(f"Tipo de exceção: {exc_type}")
            print(f"Valor da exceção: {exc_val}")
            print(f"Traceback da exceção: {exc_tb}")
        return False  # Re-raise exceptions, if any

In [79]:
def retorna_arquivos(tipo:eTipo, path:str):
    # Padrão de arquivo com máscara
    padrao = f"{tipo.nome}*.parquet"
    print(f"padrao->{padrao}")
    print(f"arquivo->{str(Path(path))}")
    # Buscar arquivos que correspondem ao padrão
    arquivos = glob.glob(str(Path(path) / padrao))
    print(f"arquivos->{arquivos}")
    # Verificar se algum arquivo foi encontrado
    return arquivos

In [80]:
arquivos = retorna_arquivos(eTipo.PROCESSAMENTO, path="../data/parquet/silver")

padrao->Processamento*.parquet
arquivo->../data/parquet/silver
arquivos->['../data/parquet/silver/Processamento.parquet']


In [81]:
arquivos

['../data/parquet/silver/Processamento.parquet']

In [57]:
from decimal import Decimal, InvalidOperation
def converter_numero_ingles_para_brasileiro_decimal(numero_str):
    try:
         # Remover o separador de milhar
        numero_str = numero_str.replace('.', '')
        # Substituir o separador decimal
        numero_str = numero_str.replace(',', '.')
        # Converter para Decimal
        return pd.to_numeric(numero_str, errors='coerce')
    except (ValueError, InvalidOperation):
        return None

In [72]:
def processa_bronze(tipo:eTipo, path:str):

    arquivos = retorna_arquivos(tipo, path)

    if arquivos:
        if (tipo in (eTipo.PRODUCAO, eTipo.COMERCIALIZACAO)):
            df = pd.read_parquet(arquivos[0])
            df_tipado = df.copy()
            df_agrupado = df_tipado.copy()
            df_detalhes = df_tipado.copy()

            # normalizar quantidade
            df_tipado["Quantidade (L.)"] = df_tipado["Quantidade (L.)"].apply(converter_numero_ingles_para_brasileiro_decimal)
            print(f"df_tipado->{df_tipado}")

            # Filtrar os dados para Produto='Total'
            df_total_ano = df_tipado[df_tipado['Produto'] == 'Total'].copy()
            print(f"df_total_ano->{df_total_ano}")
            df_total_ano = df_total_ano.groupby('ano')['Quantidade (L.)'].sum().reset_index()

            df_agrupado = df_agrupado[df_agrupado["Produto"].str.isupper()]
            df_agregado_ano = df_agrupado.groupby(['ano', 'Produto'], as_index=False).agg({'Quantidade (L.)': 'sum'})
            print(f"df_agregado_ano->{df_agregado_ano}")
            df_agregado_ano.to_parquet(f'../data/parquet/silver/{tipo.nome}_AGREGADO_ANO_CATEGORIA.parquet')

            df_detalhes['categoria'] = df_detalhes['Produto'].where(df_detalhes['Produto'].str.isupper()).ffill()
            df_detalhes = df_detalhes[~df_detalhes['Produto'].str.isupper()]
            df_detalhes = df_detalhes[df_detalhes['Produto'] != 'Total']

            print(f"df_detalhes->{df_detalhes}")
        elif(tipo == eTipo.PROCESSAMENTO):
            dfs = []
            for arquivo in arquivos:
                print(f'processando arquivo: {arquivo}')
                df = pd.read_parquet(arquivo)
                df.rename(columns={"Sem definição":"Cultivar"}, inplace=True)
                dfs.append(df)

            df = pd.concat(dfs, ignore_index=True)

            df_tipado = df.copy()
            df_agrupado = df_tipado.copy()
            df_detalhes = df_tipado.copy()

            df_tipado["Quantidade (Kg)"] = df_tipado["Quantidade (Kg)"].apply(converter_numero_ingles_para_brasileiro_decimal)
            print(f"df_tipado->{df_tipado}")
            #if (tipo.subtipo.value["nome"] != "Sem Classificação"):
                # Filtrar os dados para Produto='Total'
            df_total_ano = df_tipado[df_tipado['Cultivar'] == 'Total'].copy()                    
            df_total_ano = df_total_ano.groupby('ano')['Quantidade (Kg)'].sum().reset_index()
            print(f"df_total_ano->{df_total_ano}")

            df_agrupado = df_agrupado[df_agrupado["Cultivar"].str.isupper()]
            df_agregado_ano = df_agrupado.groupby(['ano', 'Cultivar'], as_index=False).agg({'Quantidade (Kg)': 'sum'})
            print(f"df_agregado_ano->{df_agregado_ano}")
            df_agregado_ano.to_parquet(f'../data/parquet/silver/{tipo.nome}_AGREGADO_ANO_CATEGORIA.parquet')

            df_detalhes['categoria'] = df_detalhes['Cultivar'].where(df_detalhes['Cultivar'].str.isupper()).ffill()
            df_detalhes = df_detalhes[~df_detalhes['Cultivar'].str.isupper()]
            df_detalhes = df_detalhes[df_detalhes['Cultivar'] != 'Total']
            print(f"df_detalhes->{df_detalhes}")
            #else:
                # Filtrar os dados para Produto='Total'
                #df_total_ano = df_tipado[df_tipado['Sem definição'] == 'Total'].copy()
                #print(f"df_total_ano->{df_total_ano}")
        elif (tipo in (eTipo.IMPORTACAO, eTipo.EXPORTACAO)):
            df = pd.read_parquet(arquivos[0])
            df_tipado = df.copy()
            df_agrupado = df_tipado.copy()
            df_detalhes = df_tipado.copy()

            df_total_ano = df_tipado[df_tipado['Países'] == 'Total'].copy()
            df_total_ano = df_total_ano.groupby('ano')[['Quantidade (Kg)', 'Valor (US$)']].sum().reset_index()
            print(f"df_total_ano->{df_total_ano}")

            df_tipado["Quantidade (Kg)"] = df_tipado["Quantidade (Kg)"].apply(converter_numero_ingles_para_brasileiro_decimal)
            df_tipado["Valor (US$)"] = df_tipado["Valor (US$)"].apply(converter_numero_ingles_para_brasileiro_decimal)            
            print(f"df_tipado->{df_tipado}")
            
            df_detalhes = df_detalhes[df_detalhes['Países'] != 'Total']
            print(f"df_detalhes->{df_detalhes}")

        # Somar a quantidade por ano
        df_total_ano.to_parquet(f'../data/parquet/silver/{tipo.nome}_AGREGADO_ANO.parquet')
        df_detalhes.to_parquet(f'../data/parquet/silver/{tipo.nome}.parquet')



In [64]:
processa_bronze(eTipo.PRODUCAO, path="../data/parquet/bronze")

padrao->Producao_*.parquet
arquivo->../data/parquet/bronze
arquivos->['../data/parquet/bronze/Producao_1970_2024.parquet']
df_tipado->                                       Produto  Quantidade (L.)   ano  \
0                                VINHO DE MESA      217208604.0  1970   
1                                        Tinto      174224052.0  1970   
2                                       Branco         748400.0  1970   
3                                       Rosado       42236152.0  1970   
4                VINHO FINO DE MESA (VINIFERA)       23899346.0  1970   
...                                        ...              ...   ...   
2855  Destilado alcoólico simples de bagaceira              NaN  2024   
2856                         Vinho acidificado              NaN  2024   
2857             Mosto parcialmente fermentado              NaN  2024   
2858                          Outros derivados              NaN  2024   
2859                                     Total              0.0

In [65]:
processa_bronze(eTipo.COMERCIALIZACAO, path="../data/parquet/bronze")

padrao->Comercializacao_*.parquet
arquivo->../data/parquet/bronze
arquivos->['../data/parquet/bronze/Comercializacao_1970_2024.parquet']
df_tipado->                 Produto  Quantidade (L.)   ano             tipo
0          VINHO DE MESA       98327606.0  1970  Comercializacao
1                  Tinto       83300735.0  1970  Comercializacao
2                 Rosado         107681.0  1970  Comercializacao
3                 Branco       14919190.0  1970  Comercializacao
4     VINHO FINO DE MESA        4430629.0  1970  Comercializacao
...                  ...              ...   ...              ...
3460      Vinho composto              NaN  2024  Comercializacao
3461      Vinho licoroso              NaN  2024  Comercializacao
3462          Vinho leve              NaN  2024  Comercializacao
3463   Vinho gaseificado              NaN  2024  Comercializacao
3464               Total              0.0  2024  Comercializacao

[3465 rows x 4 columns]
df_total_ano->     Produto  Quantidade (L.)   a

In [66]:
processa_bronze(eTipo.PROCESSAMENTO, path="../data/parquet/bronze")

padrao->Processamento_*.parquet
arquivo->../data/parquet/bronze
arquivos->['../data/parquet/bronze/Processamento_Sem classificação_1970_2024.parquet', '../data/parquet/bronze/Processamento_Americanas e híbridas_1970_2024.parquet', '../data/parquet/bronze/Processamento_Viníferas_1970_2024.parquet', '../data/parquet/bronze/Processamento_Uvas de mesa_1970_2024.parquet']
processando arquivo: ../data/parquet/bronze/Processamento_Sem classificação_1970_2024.parquet
processando arquivo: ../data/parquet/bronze/Processamento_Americanas e híbridas_1970_2024.parquet
processando arquivo: ../data/parquet/bronze/Processamento_Viníferas_1970_2024.parquet
processando arquivo: ../data/parquet/bronze/Processamento_Uvas de mesa_1970_2024.parquet
df_tipado->                       Cultivar  Quantidade (Kg)   ano           tipo  \
0             Sem classificação        3675463.0  1970  Processamento   
1                         Total        3675463.0  1970  Processamento   
2             Sem classificação  

In [73]:
processa_bronze(eTipo.IMPORTACAO, path="../data/parquet/bronze")

padrao->Importacao_*.parquet
arquivo->../data/parquet/bronze
arquivos->['../data/parquet/bronze/Importacao_Suco de uva_1970_2024.parquet', '../data/parquet/bronze/Importacao_Espumantes_1970_2024.parquet', '../data/parquet/bronze/Importacao_Uvas frescas_1970_2024.parquet', '../data/parquet/bronze/Importacao_Vinhos de mesa_1970_2024.parquet', '../data/parquet/bronze/Importacao_Uvas passas_1970_2024.parquet']
df_total_ano->     ano Quantidade (Kg) Valor (US$)
0   1970               0           0
1   1971             485         950
2   1972             639         311
3   1973           5.522       2.489
4   1974          10.949       3.914
5   1975           3.600         977
6   1976          18.670       5.467
7   1977               0           0
8   1978           2.790       2.850
9   1979              49          64
10  1980               0           0
11  1981               0           0
12  1982              48          99
13  1983               0           0
14  1984             

In [74]:
processa_bronze(eTipo.EXPORTACAO, path="../data/parquet/bronze")

padrao->Exportacao_*.parquet
arquivo->../data/parquet/bronze
arquivos->['../data/parquet/bronze/Exportacao_Espumantes_1970_2024.parquet', '../data/parquet/bronze/Exportacao_Uvas frescas_1970_2024.parquet', '../data/parquet/bronze/Exportacao_Vinhos de mesa_1970_2024.parquet', '../data/parquet/bronze/Exportacao_Suco de uva_1970_2024.parquet']
df_total_ano->     ano Quantidade (Kg) Valor (US$)
0   1970           5.583       3.384
1   1971             510         294
2   1972           3.467       1.581
3   1973          76.023      50.256
4   1974           8.245       6.687
5   1975          20.163      14.264
6   1976           8.307       9.400
7   1977          21.516      20.019
8   1978          10.930      13.076
9   1979           4.186       4.248
10  1980          18.036      19.717
11  1981          69.884      82.106
12  1982          78.396     123.103
13  1983          20.354      34.568
14  1984           2.443       4.984
15  1985          10.274      19.411
16  1986      